In [1]:
#Import Packages
from selenium import webdriver
import time
import requests
import pandas as pd
import os
import csv
from bs4 import BeautifulSoup

In [2]:
#Import Packages
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [203]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

In [4]:
url = "https://in.indeed.com/jobs?q=software+developer+fresher&l=&from=searchOnHP&vjk=0b6cb67749b41b47" #url for specific job

In [5]:
driver = webdriver.Chrome() #open url in chrome
driver.implicitly_wait(2)
driver.get(url)

In [127]:
d = {"Company_name":[], "Role":[] , "Job_description":[],"Salary":[], "Location":[], "Application_link":[]}
print(d)

{'Company_name': [], 'Role': [], 'Skills': [], 'Job_description': [], 'Salary': [], 'Location': [], 'Application_link': []}


In [91]:
def get_role_name(soup):
#get role 
    try:
        Job_name = soup.find('h1', class_='jobsearch-JobInfoHeader-title').span.text

    except AttributeError:
        Job_name = ""
    return Job_name

def get_company_name(soup):
    try:
        Company_name = soup.find('span', class_='css-1saizt3 e1wnkr790').text
    except AttributeError:
        Company_name = ""
    return Company_name

def get_location(soup):
    try:
        location = soup.find('div', class_='css-waniwe eu4oa1w0').text

    except AttributeError:
        location = ""
    return location

def get_jd(soup):
    try:
        job_description = soup.find('div', id='jobDescriptionText').text.strip()

    except AttributeError:
        job_description = ""
    return job_description

def get_salary(soup):
    try:
        details_span = soup.find('div', id='salaryInfoAndJobType')
        salary = details_span.text if details_span else None

    except AttributeError:
        salary = ""
    return salary

In [240]:
def get_info(driver):
    html_content = driver.page_source  # Get the HTML content of the current page
    soup = BeautifulSoup(html_content, 'html.parser') # Parse the HTML content using BeautifulSoup
    links = soup.find_all("a", attrs={'class':'jcs-JobTitle css-jspxzf eu4oa1w0'}) # Find all links with the specified class
    links_list = []
    for link in links:
        links_list.append(link.get('href'))
    for link in links_list:
        new_webpage = requests.get("https://in.indeed.com" + link, headers=HEADERS) # Send request to access the page
        new_link = "https://in.indeed.com" + link
        new_soup = BeautifulSoup(new_webpage.content, "html.parser") # Parse the HTML content of the new webpage
        time.sleep(10)
        d["Company_name"].append(get_company_name(new_soup))
        d["Role"].append(get_role_name(new_soup))
        d["Location"].append(get_location(new_soup))
        d["Salary"].append(get_salary(new_soup))
        d["Job_description"].append(get_jd(new_soup))
        d["Application_link"].append(new_link)

In [ ]:
for i in range(20):
    get_info(driver)
    next_page_button = driver.find_element(By.CSS_SELECTOR, '[data-testid="pagination-page-next"]') # Find the button to click next
    next_page_button.click() # Click button
    print("Page ",i," clicked")

In [220]:
lengths = {key: len(value) for key, value in d.items()}
print(lengths)

{'Company_name': 184, 'Role': 184, 'Job_description': 184, 'Salary': 184, 'Location': 184, 'Application_link': 184}


In [221]:
df = pd.DataFrame.from_dict(d)

  # Save DataFrame to CSV file
csv_file_path = "job_data_indeed1.csv"
df.to_csv(csv_file_path, index=False)

In [237]:
d1 = d
print(d1)

{'Company_name': ['Hyres', 'OSINT Organization', 'Indas Analytics LLP', 'Kapiital Kapslock', 'InnerUs Foundationz', 'Fortune InfoServeIndia Pvt Ltd', 'Global Technology Services', 'Snapper IT', 'Greenai', 'Chaosology', 'Francina & Francis Consulting', 'CREATIVE SEATING SYSTEMS', 'TCR Advanced Engineering Pvt Ltd', 'Blumac', 'Nj Technos', 'Vesra Software services', 'SHAMGAR SOFTWARE SOLUTIONS', 'BlueMoon', 'CYSLYS', 'Aadhar Global Software', 'Edsol Technologies ( OPC ) PRIVATE LIMITED', 'Alpha Technology And Software Solutions', 'biotic classes', 'Intralogic IT Solutions', 'AKR Textile', 'Diverse Infotech Pvt Ltd', 'CAPAAR', 'Bobsev Pvt Ltd', 'Vcheck Global', 'CodeMax IT Solutions Pvt. Ltd.', 'SaneCraft PVT LTD', 'INFRANIX Technologies', 'Unstoppable Welfare Society', 'Qualitywebs', 'AS Global Soft Tech', 'SLV Technologies Pvt Ltd', 'Vplak India Pvt ltd.', 'migister pvt ltd', 'Necxis', 'Lateshipment', 'Aara Infosystems Private Limited', 'Hyeongshin Automotive Industry', 'amdocs', 'GRABT

In [239]:
import csv

def remove_empty_rows(input_csv, output_csv, encoding='utf-8'):
    with open(input_csv, 'r', newline='', encoding=encoding) as infile:
        reader = csv.reader(infile)
        rows = list(reader)

    # Filter out rows with empty values in all columns except the last one
    filtered_rows = [row for row in rows if any(row[:-1])]

    with open(output_csv, 'w', newline='', encoding=encoding) as outfile:
        writer = csv.writer(outfile)
        writer.writerows(filtered_rows)

# Example usage
input_csv = 'job_data_indeed1.csv'
output_csv = 'indeed.csv'
remove_empty_rows(input_csv, output_csv)
